1. Вибір моделі

Використовую модель MobileNetV2 із TensorFlow Hub для класифікації зображень

In [31]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

#Завантаження натренованої моделі з TensorFlow Hub
base_model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
base_model = hub.KerasLayer(base_model_url, input_shape=(224, 224, 3), trainable=False)

#Фіксуємо початкові шари моделі
base_model.trainable = False

ValueError: Trying to load a model of incompatible/unknown type. '/var/folders/f4/gn0xg2wj5ln_fbb_v_0_2h940000gn/T/tfhub_modules/145bb06ec3b59b08fb564ab752bd5aa222bfb50a' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

2. Збір та підготовка даних

Використовую датасет CIFAR-10, який містить 10 класів зображень

In [26]:
# Завантаження CIFAR-10
from tensorflow.keras.datasets import cifar10

# Завантаження даних
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Фільтрування даних для двох класів: коти (3) і собаки (5)
classes = [3, 5]
train_filter = np.isin(y_train, classes).reshape(-1)
test_filter = np.isin(y_test, classes).reshape(-1)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Перекодування міток: 3 -> 0 (коти), 5 -> 1 (собаки)
y_train = (y_train == 5).astype(int).reshape(-1)
y_test = (y_test == 5).astype(int).reshape(-1)

# Масштабування даних
x_train = x_train / 255.0
x_test = x_test / 255.0

# Зміна розміру зображень під вимоги моделі
x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")

x_train shape: (10000, 224, 224, 3), y_train shape: (10000,)


3. Адаптація моделі

Додано новий класифікаційний шар

In [27]:
from tensorflow.keras import layers, Model
import tensorflow_hub as hub
import tensorflow as tf

# Завантаження базової моделі з TensorFlow Hub
base_model_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"
base_model = hub.KerasLayer(base_model_url, trainable=False)  # Freezing the base model

# Побудова моделі за допомогою функціональної API
inputs = layers.Input(shape=(224, 224, 3))  # Вхідні дані
x = base_model(inputs, training=False)  # Передача `training=False` для замороженої моделі
x = layers.GlobalAveragePooling2D()(x)  # Додавання шару для згладжування
x = layers.Dense(128, activation='relu')(x)  # Додатковий повнозв’язаний шар
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)  # Вихідний шар (бінарна класифікація)
model = Model(inputs, outputs)

# Компіляція моделі
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Виведення структури моделі
model.summary()

ValueError: Exception encountered when calling layer 'keras_layer_10' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_10' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_6>
  • training=False

4. Fine-tuning

In [28]:
#Розморозимо останні 50 шарів базової моделі
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

#Повторна компіляція
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

AttributeError: 'KerasLayer' object has no attribute 'layers'

5. Тренування моделі

In [29]:
history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=32,
                    verbose=1)

NameError: name 'model' is not defined

6. Оцінка моделі

In [30]:
#Оцінка моделі
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Точність на тестових даних: {test_accuracy:.2f}")

#Матриця похибок
y_pred = (model.predict(x_test) > 0.5).astype("int32").reshape(-1)
cm = confusion_matrix(y_test, y_pred)

#Візуалізація матриці похибок
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Cats', 'Dogs'], yticklabels=['Cats', 'Dogs'])
plt.title("Матриця похибок")
plt.xlabel("Прогнозовані класи")
plt.ylabel("Реальні класи")
plt.show()

#Класифікаційний звіт
print("Класифікаційний звіт:")
print(classification_report(y_test, y_pred, target_names=['Cats', 'Dogs']))

NameError: name 'model' is not defined

7. Візуалізація передбачень

In [ ]:
# Візуалізація передбачень
def display_predictions(images, labels, predictions):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        title = f"Real: {'Dog' if labels[i] else 'Cat'}\nPredicted: {'Dog' if predictions[i] else 'Cat'}"
        plt.title(title)
        plt.axis('off')
    plt.show()

# Виведення передбачень
display_predictions(x_test.numpy(), y_test, y_pred)